In [1]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 10]

In [2]:
deletion_score = -1
gap_score = -1
mismatch_score = -1
match_score = +1

In [4]:
def align(template, sequence, cache = None):
    if cache is None:
        cache = dict()
    if (template, sequence) in cache:
        return cache[(template, sequence)]
    if not template:
        return len(sequence) * deletion_score, ""
    if not sequence:
        return len(template) * gap_score, template
    
    template_head, template_tail = template[0], template[1:]
    sequence_head, sequence_tail = sequence[0], sequence[1:]
    
    if template_head == sequence_head:
        match_mismatch_score = match_score
    else:
        match_mismatch_score = mismatch_score
    
    s, seq = align(template_tail, sequence_tail, cache)
    match_result = s + match_mismatch_score, template_head + seq
    s, seq = align(template, sequence_tail, cache)
    delete_result = s + deletion_score, seq
    s, seq = align(template_tail, sequence, cache)
    gap_result = s + gap_score, template_head + seq
    
    results = [match_result, delete_result, gap_result]
    results.sort()
    result = results[-1]
    cache[(template, sequence)] = result
    return result

In [4]:
align("GCATGCT", "GATTACA")

(0, 'GCATGCT')

In [5]:
def produce_sequences(pattern, max_length, outer_index, inner_index, partial_result):
    if len(partial_result) >= max_length:
        return [partial_result]

    current_pattern = pattern[outer_index]
    results = []
    next_pattern_available = outer_index != len(pattern) - 1 and (inner_index == 0)
        
    next_inner_index = (inner_index + 1) % len(current_pattern)
    current_character = pattern[outer_index][inner_index]
    next_partial = partial_result + current_character
    
    if next_pattern_available:
        jump_right_now = produce_sequences(pattern, max_length, outer_index + 1, 0, partial_result)
        for p in jump_right_now:
            results.append(p)
    
    same_pattern_result = produce_sequences(pattern, max_length, outer_index, next_inner_index, next_partial)
    for p in same_pattern_result:
        results.append(p)
    
    return results

In [6]:
def produce_sequences_outer(pattern, max_length):
    return produce_sequences(pattern, max_length, 0, 0, "")

In [8]:
produce_sequences_outer(("AA", "BB"), 5)

['BBBBB', 'AABBB', 'AAAAB', 'AAAAA']

In [9]:
produce_sequences_outer(("CTG", "CCGCTG", "CTG"), 12)

['CTGCTGCTGCTG',
 'CCGCTGCTGCTG',
 'CCGCTGCCGCTG',
 'CTGCTGCTGCTG',
 'CTGCCGCTGCTG',
 'CTGCCGCTGCCG',
 'CTGCTGCTGCTG',
 'CTGCTGCCGCTG',
 'CTGCTGCTGCTG',
 'CTGCTGCTGCCG',
 'CTGCTGCTGCTG']

In [17]:
def produce_sequence_and_align(pattern, template):
    sequences = produce_sequences_outer(pattern, len(template))
    results = sorted([align(seq, template) for seq in sequences])
    return results[-1]

In [18]:
def collapse_sequence(pattern, sequence):
    answer = []
    repetitions = 0
    for elem in pattern:
        while True:
            tried = elem * repetitions
            if sequence[:len(tried)] != tried:
                answer.append(repetitions - 1)
                sequence = sequence[len(tried) - len(elem):]
                repetitions = 0
                break
            repetitions += 1
    return list(zip(answer, pattern))

In [19]:
def best_templates_from_raw_reads(pattern, sequences):
    avg_sequence_length = round(sum(len(sequence) for sequence in sequences)/len(sequences))
    candidate_patterns = produce_sequences_outer(pattern, avg_sequence_length)
    scores = {sequence: 0 for sequence in candidate_patterns}
    for pattern in candidate_patterns:
        for sequence in sequences:
            scores[pattern] += align(pattern, sequence)[0]
    
    for pattern in candidate_patterns:
        for sequence in sequences:
            scores[pattern] += align(pattern, sequence)[0]
    
    sorted_sequences = sorted([(value, sequence) for sequence, value in scores.items()], reverse=True)
    return sorted_sequences

# Demo on non-biological data

In [20]:
pattern = ["AA", "ABA", "BB"]

In [21]:
produce_sequences_outer(pattern, 7)

['BBBBBBB',
 'ABABBBB',
 'ABAABAB',
 'ABAABAA',
 'AABBBBB',
 'AAABABB',
 'AAABAAB',
 'AAAABBB',
 'AAAAABA',
 'AAAAAAB',
 'AAAAAAA',
 'AAAAAAA']

In [22]:
reads = ["AAABABB", "AACBABB", "AABBABB", "AAABBBB", "AAABAB"]
result = best_templates_from_raw_reads(pattern, reads)
result

[(54, 'AAABABB'),
 (38, 'AAABAAB'),
 (34, 'AABBBBB'),
 (32, 'AAAABBB'),
 (30, 'ABABBBB'),
 (30, 'ABAABAB'),
 (18, 'AAAAABA'),
 (18, 'AAAAAAB'),
 (10, 'ABAABAA'),
 (-4, 'AAAAAAA'),
 (-6, 'BBBBBBB')]

In [23]:
best_template = result[0][1]

In [24]:
best_template

'AAABABB'

In [25]:
collapse_sequence(pattern, best_template)

[(1, 'AA'), (1, 'ABA'), (1, 'BB')]

# Problems with biological data

In [21]:
hypothetical_pattern = ["CTG", "CCGCTG", "CTG"]

In [22]:
sequence = 'GCTGTTGCTGCTGCTTGCTGCTGCTTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCATGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCGCTGCTGCTGCTGCTGCTGCTGCTGCCGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGATGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCCGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCGCTGCCGCTGCCGCTGCGCTGCCGCTGCGCTGCCGCTGCCGCTGCCGCTGCCGCTGCGCTGCGCTGCCGCTGCCGCTGCCGCTGCCGCTGCGCTGCGCTGCGCTGCGCTGCCGCTGCCGCTGCCGCTGCCGCTGCCGCTGCCGCTGCCGCTGCCGCTGCGCTGCCGCTGCGCTGCGCTGCGCTGCCGCTGCGCTGCCGCTGCGCTGCCGCTGCCGCTGCCGCTGCCGCTGCGCTGCGCTGCCGCTGCCGCTGCGCTGCGCTGCCGCTGCGCTGCCGCTGCCGCTGCGCTGCCGCTGCCGCAGCCGCTGCCGCTGCCGCTGCCGCTGCCGCTGCCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCCGCTGCTGCTGCT'

In [23]:
len(sequence)

1389

In [24]:
import time
start = time.time()
candidate_templates = produce_sequences_outer(["CTG", "CTGCCG", "CTG"], len(sequence))
stop = time.time()

In [25]:
print("took {}s".format(stop - start))

took 29.27958345413208s


In [19]:
len(candidate_templates)

54288

In [20]:
start = time.time()
align(candidate_templates[0], sequence)
stop = time.time()

In [21]:
single_alignment = stop - start

In [22]:
print("took {}s".format(single_alignment))

took 19.319965600967407s


In [23]:
total_time_seconds = single_alignment * len(candidate_templates) * 700

In [24]:
total_time_seconds

734189604.781723

In [25]:
total_years = total_time_seconds/(3600*24*30*12)

In [26]:
total_years

23.604346861552308

1. Biostatisticians in Glasgow suggested I use a "Markov Change-Point model" and build a maximum liklihood model (should be faster).
2. I haven't tried it yet, but interested to collaborate!